In [471]:
import numpy as np
###################SETUP###################


numbers=(1,1,1,2,2,3,3,4,4,5)
colors=('r','y','g','b','w')           #possibly add multi... "m"
NumCol=len(colors)

def CreateDeck():
    from random import shuffle
    deck=[]
    for number in numbers:
        for color in range(NumCol):
            deck.append([number-1,color])    #internal format numbers range 0-4; colors are numbered
    shuffle(deck)
    #deck=tuple(deck)        #converts deck from list to tuple (mutable to immutable)
    return deck             #maybe keep as list if want multiple deals

#CREATE A FULL SHUFFLED DECK WITH NOMENCLATURE AS IN [2,0] FOR "3 red"
#DECK OUT CONVERTS TO EXTERNAL FORMAT.
deck=CreateDeck()
DeckOut=[str(card[0]+1)+colors[card[1]] for card in deck]
#for i in range(int(len(deck)/4)+1):          #PRINTS DECK 4 AT A TIME
    #print(DeckOut[4*i:4*(i+1)])

#THE ROWS IN THE PRIVATE MATRIX status DESCRIBE EACH PERSPECTIVE ABOUT CARDS:
# -9.00) DISCARDED TO TRASH OR UNSUCCESSFULLY PLAYED
# -8.00) SURVIVOR, UNSAVED
# -7.00) ENDANGERED TWINLESS OR DOUBLE CHOP, UNSAVED
# -1.25) DEAD (UNKNOWN TO PLAYER)
# -1.00) DEAD (KNOWN TO PLAYER)
#  0.00) IN DRAW PILE
#  0.25) MARKED (NUMBER AND/OR COLOR)
#  0.50) TWINLESS 4 (NOT ENDANGERED)
#  0.75) TWINLESS 3 (NOT ENDANGERED)
#  1.00) EXPECTED TO PLAY IN QUEUE
#  1.25) EXPECTED TO PLAY NEXT (NOT 2.50)
#  2.50) EXPECTED TO IGNITE CHAIN (UNMARKED)
#  9.00) PLAYED TO STACK
status=np.zeros((5,len(deck)))

#THE PRIVATE ARRAY info HOLDS THE PERSPECTIVES OF EACH OF THE FIVE PLAYERS.
#COORDINATES (i,j,k) POINT TO AN 8 BIT STRING CORRESPONDING TO INFORMATION
#THAT PLAYER i KNOWS ABOUT PLAYER j'S HAND WITH RESPECT TO CLUE k. THE FIRST
#4 BITS HAVE NEGATIVE INFORMATION FOR THE CORRESPONDING 4 CARDS IN PLAYER j'S
#HAND. THE LAST 4 BITS HAVE POSITIVIE INFORMATION.
###################################info=np.ndarray(shape=(5,5,10), dtype=np.uint8, order='C')         #For 6 colors, need shape=5,11

#THE PRIVATE ARRAY metainfo HOLDS METAINFORMATION ABOUT EACH HAND, SIMILAR TO info.
#COORDINATES (i,j,k,l) DESCRIBE IN THE SAME WAY AS info WHAT PLAYER i THINKS
#PLAYER j KNOWS ABOUT PLAYER k'S HAND WITH RESPECT TO CLUE l.
#####################################metainfo=np.array(shape=(5,5,5,10), dtype=np.uint8, order='C')

#THE PUBLIC LIST chop TELLS WHICH CARD IS UP FOR DISCARD
chop=[0,4,8,12,16]

#THE PUBLIC LISTS front DESCRIBES THE OVERALL ORDER THAT PLAYERS PRIORITIZE THEIR CARDS.
#IT STARTS PRIORITIZING THE LAST CARD, WHICH IS NORMALLY THE NEWEST.
front=[0]*len(deck)
front[3]=front[7]=front[11]=front[15]=front[19]=1

#THE PUBLIC LISTS stacks AND trash SHOW THE CURRENT PROGRESS. stacks GIVES THE LAST NUMBER SUCCESSFULLY PLAYED
#FOR EACH COLOR AND trash LISTS THE DISCARD PILE FOR EACH COLOR. FOR EXAMPLE, A 5 COLOR GAME MIGHT LOOK LIKE
#stacks=[2,2,1,0,3] and trash=[[1,1,4],[3],[],[3,4],[]]. 
stacks=[0]*NumCol
trash=np.chararray((NumCol,5))

#CREATE A MATRIX OF FIVE HANDS, ONE ON EACH ROW WITH CARDS REPRESENTED BY SERIAL NUMBERS FROM
#THE ORIGINAL DECK, TOP TO BOTTOM, STARTING AT 0. WE DEAL EACH HAND FROM THE TOP OF THE DECK.
table=np.arange(20).reshape(5,4)
[A,B,C,D,E]=table



#TEST 5 COLOR CASE
def TEST():
    global deck, status, stacks, front
    deck=[[4,0],[2,3],[0,1],[0,1],[1,0],[0,2],[3,2],[2,0],[0,4],[0,4],[3,3],[1,2],[4,1],[3,0],[2,0],[1,4],[0,0],[2,2],[3,4],[0,2]]+[[0,0]]*10
    stacks=[2,0,0,0,0]
    status=np.zeros((5,len(deck)))      #status edit?
    hands=table[:4]
    #print(hands)
    
    #print(status)
    
    DeckOut=[str(card[0]+1)+colors[card[1]] for card in deck]
    
    #stacks=[]
    
    for i in range(5):
        print(DeckOut[4*i:4*(i+1)])

In [482]:
###################SEARCH FUNCTIONS###################
def CanSignal(hand, card):
    candy, place=[], 3
    while 1:                                            #checks if number can signal
        if place==card:                                 #distinguish chop save?
            candy.append(deck[hand[card]][0])
            break
        elif deck[hand[place]][0]==deck[hand[card]][0]:
            break
        else:
            place=place-1
    place=3
    while 1:                                            #checks if color can signal
        if place==card:
            candy.append(deck[hand[card]][1])
            break
        elif deck[hand[place]][1]==deck[hand[card]][1]:
            break
        else:
            place=place-1
    return candy

def FindFront(hands, front, number, color):
    candy=()                 #short for candidate
    for i in range(4):       #loop over hands
        for j in [3,2,1,0]:  #loop over cards in each hand (starting from new side)
            if deck[hands[i,j]]==[number,color] and front[hands[i,j]]>0:
                candy=(i,j)
    return candy

def Find(hands, number, color):
    candy=[]                 #candy=[-1]*len(hands)    #short for candidate
    for i in range(4):       #loop over hands
        for j in [3,2,1,0]:   #loop over cards in each hand
            if deck[hands[i,j]]==[number,color] and CanSignal(hands[i], j):
                candy.append((i,j))
                break
    return candy

def FindChain(hands, stacks, front, color):
    number=stacks[color]
    gather=[]
    while 1:
        next=FindFront(hands, front, number, color)
        if next==():
            break
        gather.append(next)
        number+=1
    next=Find(hands, number, color)
    if next: return gather+[next[-1]]    #Need fancier process to decide how to ignite
    else: return gather

def CanSkip(hands, player):
    candy=[]
    for PlayerSkip in range(4):
        if status[player,chop[PlayerSkip]]<0:
            candy.append(PlayerSkip)
            continue
        if -1 in [status[player,i] for i in hands[PlayerSkip]]:
            candy.append(PlayerSkip)
            continue
    return set(candy)

def BestChain(player):
    record=-1
    choice=[]
    hands=np.vstack((table[player+1:5], table[0:player]))
    print(hands)
    for color in range(5):
        chain=FindChain(hands, stacks, front, color)           #public lists like front should be global?
        involved={card[0] for card in chain}                  #set of involved players in chain (except for card to be clued)
        if chain and set(range(chain[0][0]))<=involved | CanSkip(hands, player):
                NumPlayed=0
                for (i,j) in chain:
                    if status[player,hands[i,j]]<1:NumPlayed=NumPlayed+1
                if NumPlayed>record:                         #TO DO: BREAK TIES PAST NUMPLAYED
                    record=NumPlayed
                    choice=chain
    return choice